# Data analysis of product_info table

In [1]:
# -*- coding: utf-8 -*

# import xlrd
import pymysql
import pandas as pd
import plotly.plotly as py
from plotly.graph_objs import *
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False

matplotlib.style.use('ggplot')
%matplotlib inline

In [2]:
# Connect to the database and read the data

print("Connect to the database-----")
db = pymysql.connect(host="39.105.165.114", user="root", password="zym2112!", use_unicode=True, charset="utf8")
print("Connect - Success!")
cursor = db.cursor()
cursor.execute("USE product_info_db;")
cursor.execute("select * from product_info; ")
rows = cursor.fetchall()
df = pd.DataFrame([[ij for ij in i] for i in rows])
print("Data read complete!")

Connect to the database-----
Connect - Success!
Data read complete!


In [3]:
db.close()
cursor.close()

## 1. Delete duplicate

In [5]:
linenum = df.shape[0]
print("Row num of data: " + str(linenum))
df_noDup = df.drop_duplicates([0])
linenum_noDup = df_noDup.shape[0]
print("Remove duplicates in column barcode")
print("Removed" + str(linenum - linenum_noDup) + " rows，there are now " + str(linenum_noDup) + " rows")
df_noDup.rename(columns={0:'barcode',1:'sellType',2:'wholesalePrice',3:'productName',4:'referencePurchasePrice',5:'retailPrice',6:'unit',7:'specification',8:'lowestRetailPrice',9:'productNature',10:'warrantyPeriod',11:'distributionMethod',12:'estimatedDaysOfUse',13:'grossProfitMargin'},
          inplace=True);
df_noDup.head()

Row num of data: 8218
Remove duplicates in column barcode
Removed14 rows，there are now 8204 rows


c:\users\ww\appdata\local\programs\python\python36\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,barcode,sellType,wholesalePrice,productName,referencePurchasePrice,retailPrice,unit,specification,lowestRetailPrice,productNature,warrantyPeriod,distributionMethod,estimatedDaysOfUse,grossProfitMargin
0,6937841780522,只销,20.0,None,20.00,35.0,瓶,500ML,NaN,0、普通商品,3650.0,购销,NaN,0.429
1,6927061500215,只销,20.0,None,1.30,2.0,个,46G,NaN,0、普通商品,180.0,购销,NaN,0.350
2,8412253001086,只销,20.0,#贝蒂斯特初级榨橄榄油500ml,72.98,92.0,瓶,500ml,NaN,0、普通商品,NaN,购销,NaN,0.207
3,8412253900105,只销,20.0,#贝蒂斯特级初榨橄榄油（礼盒）750ml*2,212.38,269.0,盒,750ml,NaN,0、普通商品,NaN,购销,NaN,0.210
4,6914973303058,只销,20.0,#德芙精选07金色装160g,67.00,78.0,盒,160g,0.0,0、普通商品,360.0,购销,0.0,0.141


In [6]:
print("Correlation coefficient of referencePurchasePrice and grossProfitMargin: " + str(df_noDup['referencePurchasePrice'].corr(df_noDup['grossProfitMargin'])))
print("Correlation coefficient of retailPrice and grossProfitMargin: " + str(df_noDup['retailPrice'].corr(df_noDup['grossProfitMargin'])))

Correlation coefficient of referencePurchasePrice and grossProfitMargin: -0.10056620520605113
Correlation coefficient of retailPrice and grossProfitMargin: 0.06895812581705467


## 2. Detect and deal with unreasonalbe data

In [8]:
df_noDup.describe()

,wholesalePrice,referencePurchasePrice,retailPrice,lowestRetailPrice,warrantyPeriod,estimatedDaysOfUse,grossProfitMargin
count,8204.0,8204.000000,8204.000000,3889.0,3668.00000,3889.0,8204.000000
mean,20.0,14.980729,79.854896,0.0,534.75627,0.0,0.268086
std,0.0,139.552207,5524.461942,0.0,870.89722,0.0,0.113759
min,20.0,0.000000,0.000000,0.0,2.00000,0.0,0.000000
25%,20.0,2.200000,3.000000,0.0,300.00000,0.0,0.203000
50%,20.0,4.000000,5.800000,0.0,360.00000,0.0,0.250000
75%,20.0,9.000000,12.500000,0.0,365.00000,0.0,0.308000
max,20.0,8000.000000,500239.000000,0.0,10000.00000,0.0,1.000000


In [ ]:
# From the table we can find that:
# The values of the wholesalePrice column are all 20, which is a problem；
# The maxValues in column muchreferencePurchasePrice and retailPrice are very large, and we suspect that there exists abnormal data.ta
# The values in column estimatedDaysOfUse are all 0, which is also a problem.
# Here we will further analyze the suspect data below: 

In [10]:
# referencePurchasePrice
df_noDup[df_noDup['referencePurchasePrice']>1000]


,barcode,sellType,wholesalePrice,productName,referencePurchasePrice,retailPrice,unit,specification,lowestRetailPrice,productNature,warrantyPeriod,distributionMethod,estimatedDaysOfUse,grossProfitMargin
7986,8180500100515,只销,20.0,资产冰柜（伊利SD-182）专,2000.0,2000.0,台,None,0.0,0、普通商品,NaN,购销,0.0,0.0
7997,8180500100218,只销,20.0,资产电脑主机,2000.0,2000.0,台,None,0.0,0、普通商品,NaN,购销,0.0,0.0
7999,8180500100379,只销,20.0,资产发布机,5000.0,5000.0,台,None,0.0,0、普通商品,NaN,购销,0.0,0.0
8006,8180500100409,只销,20.0,资产咖啡饮料机,2000.0,2000.0,台,None,0.0,0、普通商品,NaN,购销,0.0,0.0
8010,8180500100089,只销,20.0,资产空调壁挂,2800.0,2800.0,台,None,0.0,0、普通商品,NaN,购销,0.0,0.0
8011,8180500100119,只销,20.0,资产空调吸顶,8000.0,8000.0,个,None,0.0,0、普通商品,NaN,购销,0.0,0.0
8055,8180500100386,只销,20.0,资产液晶电视（广告）,2000.0,2000.0,台,None,0.0,0、普通商品,NaN,购销,0.0,0.0
8061,8180500100157,只销,20.0,资产中澳冷柜(陈列饮料,6000.0,6000.0,台,None,0.0,0、普通商品,NaN,购销,0.0,0.0


In [11]:
# It can be found that the higher-priced products are expensive household appliances, whose price is not abnormal under this circumstance.              

In [12]:
# retailPrice
df_noDup[df_noDup['retailPrice']>1000]


,barcode,sellType,wholesalePrice,productName,referencePurchasePrice,retailPrice,unit,specification,lowestRetailPrice,productNature,warrantyPeriod,distributionMethod,estimatedDaysOfUse,grossProfitMargin
1406,6936501587259,只销,20.0,百灵双喜缎方格毛巾,5.0,500239.0,条,None,NaN,0、普通商品,NaN,购销,NaN,1.0
7986,8180500100515,只销,20.0,资产冰柜（伊利SD-182）专,2000.0,2000.0,台,None,0.0,0、普通商品,NaN,购销,0.0,0.0
7997,8180500100218,只销,20.0,资产电脑主机,2000.0,2000.0,台,None,0.0,0、普通商品,NaN,购销,0.0,0.0
7999,8180500100379,只销,20.0,资产发布机,5000.0,5000.0,台,None,0.0,0、普通商品,NaN,购销,0.0,0.0
8006,8180500100409,只销,20.0,资产咖啡饮料机,2000.0,2000.0,台,None,0.0,0、普通商品,NaN,购销,0.0,0.0
8010,8180500100089,只销,20.0,资产空调壁挂,2800.0,2800.0,台,None,0.0,0、普通商品,NaN,购销,0.0,0.0
8011,8180500100119,只销,20.0,资产空调吸顶,8000.0,8000.0,个,None,0.0,0、普通商品,NaN,购销,0.0,0.0
8055,8180500100386,只销,20.0,资产液晶电视（广告）,2000.0,2000.0,台,None,0.0,0、普通商品,NaN,购销,0.0,0.0
8061,8180500100157,只销,20.0,资产中澳冷柜(陈列饮料,6000.0,6000.0,台,None,0.0,0、普通商品,NaN,购销,0.0,0.0


In [13]:
# It can be found that the retailprice of "百灵双喜缎方格毛巾" 500239.0 is absolutely incorrect, and here we delete it. 
df_noDup = df_noDup[df_noDup['retailPrice'] < 500000]

In [14]:
df_noDup[df_noDup['warrantyPeriod']>8000]

,barcode,sellType,wholesalePrice,productName,referencePurchasePrice,retailPrice,unit,specification,lowestRetailPrice,productNature,warrantyPeriod,distributionMethod,estimatedDaysOfUse,grossProfitMargin
429,6948119100065,只销,20.0,*猫王16LED应急灯充电手电筒,15.53,25.9,个,None,NaN,0、普通商品,10000.0,购销,NaN,0.400
430,6948119100041,只销,20.0,*猫王8LED应急灯充电手电筒,12.83,21.4,个,None,NaN,0、普通商品,10000.0,购销,NaN,0.401
431,6948119100096,只销,20.0,*猫王多功能好易拍K003,15.00,19.9,个,None,NaN,0、普通商品,10000.0,购销,NaN,0.246
432,6948119100607,只销,20.0,*猫王儿童安全插座B001b,36.99,61.7,个,None,NaN,0、普通商品,10000.0,购销,NaN,0.400
433,6948119100621,只销,20.0,*猫王儿童安全插座B002b,42.53,70.9,个,None,NaN,0、普通商品,10000.0,购销,NaN,0.400
434,6948119100713,只销,20.0,*猫王儿童安全插座B007a,34.43,57.4,个,None,NaN,0、普通商品,10000.0,购销,NaN,0.400
480,6908509901023,只销,20.0,*莫高蓝带冰酒500ml,168.00,258.0,瓶,500ml*6,NaN,0、普通商品,10000.0,购销,NaN,0.349
481,6908509030075,只销,20.0,*莫高水晶国宴冰酒500ml,128.00,198.0,瓶,500ml*6,NaN,0、普通商品,10000.0,购销,NaN,0.354
482,6908509900866,只销,20.0,*莫高小冰白冰酒375ml,188.00,288.0,瓶,375ml*6,NaN,0、普通商品,10000.0,购销,NaN,0.347
483,6908509900880,只销,20.0,*莫高小冰红冰酒375ml,188.00,288.0,瓶,375ml*6,NaN,0、普通商品,10000.0,购销,NaN,0.347


## 3. Data analysis

### Some statistics about the data in the table.

In [16]:
df_noDup.describe()

,wholesalePrice,referencePurchasePrice,retailPrice,lowestRetailPrice,warrantyPeriod,estimatedDaysOfUse,grossProfitMargin
count,8203.0,8203.000000,8203.000000,3889.0,3668.00000,3889.0,8203.000000
mean,20.0,14.981946,18.882186,0.0,534.75627,0.0,0.267997
std,0.0,139.560670,141.087535,0.0,870.89722,0.0,0.113479
min,20.0,0.000000,0.000000,0.0,2.00000,0.0,0.000000
25%,20.0,2.200000,3.000000,0.0,300.00000,0.0,0.203000
50%,20.0,4.000000,5.800000,0.0,360.00000,0.0,0.250000
75%,20.0,9.000000,12.500000,0.0,365.00000,0.0,0.308000
max,20.0,8000.000000,8000.000000,0.0,10000.00000,0.0,0.999000


In [17]:
print("Connect to the database-----")
db = pymysql.connect(host="39.105.165.114", user="root", password="zym2112!", use_unicode=True, charset="utf8")
print("Connect - Success!")
cursor = db.cursor()
cursor.execute("USE product_info_db;")

Connect to the database-----
Connect - Success!


0

In [18]:
cursor.execute("select count(distinct(unit)) from product_info; ")
result = cursor.fetchall()
df_2 = pd.DataFrame([[ij for ij in i] for i in result])
print("There are " + str(df_2[0][0]) + " kinds of units in the table, they are: ")
cursor.execute("select distinct(unit) from product_info; ")
result = cursor.fetchall()
df_3 = pd.DataFrame([[ij for ij in i] for i in result])
for e in df_3[0]:
    print(e,  end=', ')

There are 64 kinds of units in the table, they are: 
瓶, 个, 盒, 提, 包, 罐, 条, 支, 双, 件, 袋, 把, 张, 本, 块, 卷, 根, 杯, 听, 粒, 面, 桶, 组, 19, 板, 排, 筒, 500g, None, 串, 台, 捆, 枚, 套, kg, 碗, 大包, 辆, 只, 被, 片, 卡, 箱, 份, 次, 嗲, 架, 博爱, JIAN, 盏, 打, 付, 扎, 碟, 对, 带, 米, 平, 盒、, bao, 部, 元, 度, 各, 200g, 

In [19]:
cursor.execute("select distinct(barcode), productName, retailPrice, grossProfitMargin from product_info where retailPrice>12.5 and retailPrice<500000 and grossProfitMargin>0.308 order by grossProfitMargin DESC; ")
result = cursor.fetchall()
df_3 = pd.DataFrame([[ij for ij in i] for i in result])
print("根据retailPrice和grossProfitMargin大于75%的标准，这里筛选了 " + str(len(df_3[0])) + " 种商品，建议超市在下一阶段增大这些商品的促销力度，它们为：")
df_3

根据retailPrice和grossProfitMargin大于75%的标准，这里筛选了 522 种商品，建议超市在下一阶段增大这些商品的促销力度，它们为：


,0,1,2,3
0,6925303723644,统一桂花清茶500ML,25.0,0.919
1,6925303723651,统一茉莉清茶500ML,25.0,0.919
2,8150303800004,康力牌含氯消毒液,14.0,0.898
3,6938586493029,与狼共舞（DWP65517-5）,159.0,0.750
4,3344398004408,塔乐丝干红葡萄酒750ml,108.0,0.741
5,8437002040314,富仕阁精装型CHARDONNAY霞多丽干白750ml,228.0,0.737
6,8437002040338,富仕阁精装型CABERNETSAUVIGNON赤霞珠干红750ml,228.0,0.737
7,3077051201493,郁金香古堡干红葡萄酒750ml,158.0,0.734
8,3191740401299,维尼亚AOC干白750ml,158.0,0.734
9,8437002040321,富仕阁精装型TEMPRANILLO丹普兰尼洛干红750ml,185.0,0.730


In [20]:
db.close()
cursor.close()

### Distribution of some data

In [22]:
fig = plt.figure()
ax1= fig.add_subplot(1,2,1)
ax2= fig.add_subplot(1,2,2)
plt.rcParams['savefig.dpi'] = 1200 #图片像素
plt.rcParams['figure.dpi'] = 1200

df_noDup['retailPrice'].plot(ax=ax1,title="Distribution of retailPrice（零售价）",kind="box",figsize=(14,7))
df_noDup['grossProfitMargin'].plot(ax=ax2,title="Distribution of grossProfitMargin（毛利率）",kind="box",figsize=(14,7))
plt.show()
plt.savefig("figures/boxPlot.png")

<Figure size 7200x4800 with 0 Axes>

In [23]:
fig2 = plt.figure()
ax3= fig2.add_subplot(2,2,1)
ax4= fig2.add_subplot(2,2,2)
ax5= fig2.add_subplot(2,2,3)
plt.rcParams['savefig.dpi'] = 1200 #图片像素
plt.rcParams['figure.dpi'] = 1200

df_noDup.groupby('productNature')['barcode'].count().plot(ax=ax3,title="Distribution of productNature（商品性质）",kind="pie",figsize=(14,14))
df_noDup.groupby('sellType')['barcode'].count().plot(ax=ax4,title="Distribution of sellType（商品品态）",kind="pie",figsize=(14,14))
df_noDup.groupby('distributionMethod')['barcode'].count().plot(ax=ax5,title="Distribution of distributionMethod（经销方式）",kind="pie",figsize=(14,14))

plt.show()
plt.savefig("figures/piePlot.png")

<Figure size 7200x4800 with 0 Axes>